Copyright 2023 The TensorFlow Authors.


In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# CNN 일일 데이터세트를 사용하는 대규모 언어 모델용 TFX 파이프라인 튜토리얼

이 코드랩에서는 KerasNLP를 사용하여 사전 훈련된 대규모 언어 모델(LLM)인 GPT-2 모델을 로드하고 데이터세트에 맞게 미세 조정합니다. 이 데모에 사용하는 데이터세트는 CNN 일일 데이터세트입니다.  여기서는 엔드 투 엔드 프로세스를 보여주기 위해 GPT-2를 사용했으며, 이 코드랩에 소개된 기술과 툴링은 Google T5와 같은 다른 생성형 언어 모델에도 적용할 수 있습니다.

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/CSV_Downloader_Component"> <img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/tfx/CSV_Downloader_Component.ipynb"> <img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/tfx/CSV_Downloader_Component.ipynb"> <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tfx/tutorials/tfx/CSV_Downloader_Component.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a></td>
</table></div>

# 시작하기 전에

Colab은 다양한 종류의 런타임을 제공합니다. GPT-2 모델을 미세 조정할 것이므로 **런타임 -&gt; 런타임 변경** 유형으로 이동하여 GPU 하드웨어 가속기 런타임(12G 이상의 시스템 RAM과 15G 이하의 GPU RAM이 있어야 함)을 선택합니다.

# 설정하기

먼저 TFX Python 패키지를 설치합니다.

## Pip 업그레이드하기

로컬에서 실행할 때 시스템에서 Pip를 업그레이드하지 않으려면 Colab에서 실행 중인지 확인해야 합니다. 물론 로컬 시스템은 별도로 업그레이드할 수 있습니다.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

## TFX 설치하기

현재 Colab에서 실행되는 Python 3.10에서 TFX 관련 문제가 발생하고 있습니다. 이러한 경우 다음 명령을 실행하기만 하면 됩니다.

```
!pip install -U tfx
```

TFX 설치에 **실패할 것입니다**. 그러면 다음 코드를 따라서 작업을 수행해봅니다.

In [ ]:
%%shell
update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 3
curl -O https://bootstrap.pypa.io/get-pip.py
python get-pip.py

In [ ]:
# 1) TFX relies on an old version of google-api-core so we let google-auth float
# for the install. We grep it out below:
!grep -v google-auth /etc/requirements.core.in > requirements.txt

# 2) httplib2 should be included in /etc/requirements.core.in but it's not for
# reasons. We ensure it's included:
!grep httplib2 /etc/requirements.user.in >> requirements.txt

# 3) google.colab package is not available as a wheel. We symlink that in so
# it's on the sys.path of Python 3.8:
!mkdir /usr/local/lib/python3.8/dist-packages/google
!ln -s /usr/local/lib/python3.10/dist-packages/google/colab /usr/local/lib/python3.8/dist-packages/google/colab

# Now with those pre-requisites out of the way:
!pip install tfx==1.13.0 -r requirements.txt

In [ ]:
!pip install keras_nlp

# 가져오기

우선 처리된 가져오기 항목을 가져옵시다.

In [ ]:
from tensorflow import keras
from tfx.types import Channel
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

## shapely 설치 제거하기

TODO(b/263441833) ImportError를 피하는 임시 솔루션입니다. 다른 추가 종속성을 제거하는 대신 최신 버전의 Bigquery를 지원하여 처리하는 것이 이상적입니다.


In [ ]:
!pip uninstall shapely -y

## 런타임을 다시 시작했습니까?

Google Colab을 사용하는 경우, "런타임 다시 시작" 버튼을 클릭하거나 "런타임 &gt; 렁타임 다시 시작..." 메뉴를 사용하여 런타임을 다시 시작해야 합니다. 이는 Colab이 패키지를 로드하는 방식 때문입니다.

TensorFlow 및 TFX 버전을 확인합니다.

라이브러리 버전을 확인해 보겠습니다.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

## 변수 설정하기

파이프라인을 정의하는 데 사용되는 변수가 몇 가지 있습니다. 이러한 변수를 원하는 대로 사용자 정의할 수 있습니다. 기본적으로 파이프라인의 모든 출력은 현재 디렉터리 아래에 생성됩니다.

# CSV 다운로더

파이프라인을 보다 효율적으로 만들고 자동화가 가능하게 하려면 다운로드할 CSV 파일에 대한 다운로드 링크를 받는 구성 요소가 있는 것이 좋습니다. 추가적으로 TFX 프로덕션 ML 파이프라인의 중요한 목표 중 하나는 파이프라인 구성 요소, 실행 및 결과 아티팩트에 대한 정보가 포함된 메타데이터를 수집하는 것입니다. 즉, 메타데이터의 목적은 파이프라인 구성 요소의 계보를 분석하고 문제를 디버그하는 것이며, CSV 다운로더 구성 요소는 사용자가 데이터의 출처와 파이프라인에 들어가기 전에 데이터가 거친 전처리 단계에 대한 정보를 기록하고 추적하는 데 도움이 될 것입니다. 이 섹션에서는 CSVdoc이라는 새로운 아티팩트를 선언하고 데이터세트에 대한 정보를 저장하고 CSVdoc 아티팩트의 URI에 CSV 파일을 다운로드하는 사용자 정의 구성 요소인 CSV Downloader를 개발합니다.

In [ ]:
from tfx.types import artifact
from tfx import types

Property = artifact.Property
PropertyType = artifact.PropertyType

URL_PROPERTY = Property(type=PropertyType.STRING)
PATH_PROPERTY = Property(type=PropertyType.STRING)

class CsvDoc(types.Artifact):
  """ Artifact that contains the CSV dataset.

     - 'url' : saves the source of the original data.
     - 'path': saves the path to the CSV file.
  """

  TYPE_NAME = 'CsvDoc'
  PROPERTIES = {
      'url' : URL_PROPERTY,
      'path': PATH_PROPERTY,
  }

In [ ]:
from absl import logging
import requests
import os
import tfx.v1 as tfx
from tfx.dsl.component.experimental.decorators import component

@tfx.dsl.components.component
def CsvDownloaderComponent(
    url: tfx.dsl.components.Parameter[str],
    file_name: tfx.dsl.components.Parameter[str],
    saved_file: tfx.dsl.components.OutputArtifact[CsvDoc],
) -> None:
  response = requests.get(url)
  saved_file.url = url
  if response.status_code == 200:
    file_path = os.path.join(saved_file.uri, file_name)
    saved_file.path = file_path
    url_content = response.content
    with open(file_path, 'wb') as csv_file:
      csv_file.write(url_content)
    logging.info(f"CSV file saved successfully at {file_path}")
  else:
    raise Exception("CSV file failed to be saved.")

In [ ]:
downloader = CsvDownloaderComponent(
  url = 'https://drive.google.com/uc?id=1YdZsJlRafqxiNSl0nHQkwR7rzrNlN9LI&export=download', file_name ='testing_doc.csv')

In [ ]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
context = InteractiveContext()

In [ ]:
context.run(downloader, enable_cache = False)